# Back Orders
En este script se crean los reportes de Back Orders como Paul los necesita

In [35]:
import os
import re
import pandas as pd
import numpy as np
import openpyxl as xl
from datetime import datetime
from excel_functions import ExcelFunctions
from openpyxl.styles import PatternFill, Font
from sklearn.preprocessing import MinMaxScaler

In [2]:
def hex_to_RGB(hex_str):
    """ #FFFFFF -> [255,255,255]"""
    #Pass 16 to the integer function for change of base
    return [int(hex_str[i:i+2], 16) for i in range(1,6,2)]

In [3]:
def get_color_gradient(c1, c2, n):
    """
    Given two hex colors, returns a color gradient
    with n colors.
    """
    assert n > 1
    c1_rgb = np.array(hex_to_RGB(c1))/255
    c2_rgb = np.array(hex_to_RGB(c2))/255
    mix_pcts = [x/(n-1) for x in range(n)]
    rgb_colors = [((1-mix)*c1_rgb + (mix*c2_rgb)) for mix in mix_pcts]
    return ["".join([format(int(round(val*255)), "02x") for val in item]) for item in rgb_colors]

In [4]:
def encontrar_numeros_en_cadena(cadena):
    numeros_encontrados = re.findall(r'-?\d+(?:\.\d+)?', cadena)
    return numeros_encontrados

In [5]:
def encontrar_maximo_datetime(lista_datetime) -> int:
    max_datetime, max_index = None, None

    for i, dt in enumerate(lista_datetime):
        if max_datetime is None or dt > max_datetime:
            max_datetime = dt
            max_index = i

    return max_index

In [6]:
def encontrar_archivo_mas_reciente(nombres_archivos: list, dia_primero: bool=True):
    '''
    Esta función recibe una lista de nombres de fechas que contienen una fecha,
    y regresa el nombre de archivo con la fecha más reciente. 
    Funciona bien con los nombres de inventarios y transferencias.
    Para forecasts names usar encontrar_forecast_mas_reciente()
    '''
    fechas = []
    
    for file in nombres_archivos:
        numeros_encontrados = encontrar_numeros_en_cadena(file)
        
        if len(numeros_encontrados) != 3:
            print(f'No se pudo identificar la fecha en: {file}')
            continue
            
        fecha_encontrada = '/'.join(numeros_encontrados)
        fechas.append(pd.to_datetime(fecha_encontrada, dayfirst=dia_primero))
        
    max_date_index = encontrar_maximo_datetime(fechas)
    
    if max_date_index is not None: # Si es válido el índice lo regresamos
        return nombres_archivos[max_date_index]
    
    print('No se encontraron fechas válidas en ningún archivo')
    return None

In [7]:
def encontrar_forecast_mas_reciente(lista_archivos: list):
    '''
    Esta función recibe nombres de archivos de forecasts y regresa 
    aquel que contenga el mes y año más recientes.
    '''
    max_fecha = None
    archivo_mas_reciente = None

    for nombre_archivo in lista_archivos:
        match = re.findall(r'(\w+)_(\w+)_(\d{4})_(\w+)_(\d{4})', nombre_archivo)
        if match:
            mes_inicio = match[0][1]
            ano_inicio = int(match[0][2])
            mes_fin = match[0][3]
            ano_fin = int(match[0][4])

            fecha_inicio = datetime.strptime(f"{mes_inicio} {ano_inicio}", "%B %Y")
            fecha_fin = datetime.strptime(f"{mes_fin} {ano_fin}", "%B %Y")

            if max_fecha is None or fecha_fin > max_fecha:
                max_fecha = fecha_fin
                archivo_mas_reciente = nombre_archivo

    return archivo_mas_reciente

In [96]:
def asigna_color_celda(x, color_inicio='#ffd60a', color_fin='#d00000', columna='porcentaje'):
    # Se tiene una celda que se llama porcentaje
    # >= 0 -> no hay problema, tengo existencias para dos meses
    # < 0 -> hay problema, no tengo suficiente para los siguientes dos meses
    if x.porcentaje >= 0:
        return 'ffffff'
    colores = get_color_gradient(color_inicio, color_fin, 10)
    color_index = int(np.round(x[columna]*-10)-1)
    return colores[color_index]

In [9]:
def obtener_contraste(x):
    color_hex = x.color
    r = int(color_hex[0:2], 16)
    g = int(color_hex[2:4], 16)
    b = int(color_hex[4:6], 16)
    
    luminance = (0.299 * r + 0.587 * g + 0.114 * b) / 255
    
    if luminance > 0.5:
        return "000000"
    else:
        return "ffffff"

In [ ]:
ef = ExcelFunctions()

dir_forecast = '../transformed_data/forecasts/'
dir_inventarios = '../transformed_data/inventarios/'
dir_transfers = '../transformed_data/transferencias/'

latest_forecast = encontrar_forecast_mas_reciente(os.listdir(dir_forecast))
latest_inventario = encontrar_archivo_mas_reciente(os.listdir(dir_inventarios))
latest_transfer = encontrar_archivo_mas_reciente(os.listdir(dir_transfers))

forecast = pd.read_excel(f'{dir_forecast}/{latest_forecast}').drop(['descripcion'], axis=1)
inventario = pd.read_excel(f'{dir_inventarios}/{latest_inventario}')[['sku', 'inventario']]
transfers = pd.read_excel(f'{dir_transfers}/{latest_transfer}')[['sku', 'total']]

clasificaciones = pd.read_excel('../static_data/clasificacion_abc.xlsx').drop(['planta'], axis=1)

# Transformamos el forecast y guardamos los nombres de las columnas
forecast = (forecast
 .set_index('sku')
 .assign(total_forecast=lambda df: df.sum(axis=1))
)

file_name = f"reporte_back_order_{datetime.now().date().strftime('%d_%m_%Y')}"

In [ ]:
# Sumamos las transferencias y los inventarios por producto
output_order = ['sku', 'descripcion', 'clasificacion', 'inventario', 'transferencias', 'existencias'] + list(forecast.columns)
threshold = .2

reporte = (transfers
 .rename(columns={'total':'transferencias'})
 .merge(inventario, on='sku', how='outer')
 .merge(forecast, on='sku', how='left')
 .merge(clasificaciones, on='sku', how='left')
 # Quitamos los productos que no queremos
 .query('inventario > 0 and (clasificacion == "A" or clasificacion == "B") and total_forecast > 0') #
 # Sumamos inventarios y transferencias
 .fillna({'transferencias':0})
 .assign(
     existencias=lambda x: x.transferencias + x.inventario,
     transferencias=lambda x: x.transferencias.replace(0, np.nan),
     porcentaje=lambda x: x.existencias / x.total_forecast - 1
 )
 .sort_values('porcentaje')
 [output_order]
)

# El color de este criterio es escacez relativa
colores = (reporte
 .assign(
     porcentaje=lambda x: x.existencias / x.total_forecast - 1, # Mientras más cercano a -1 es más urgente 
     color=lambda x: x.apply(asigna_color_celda, axis=1),
     texto=lambda x: x.apply(obtener_contraste, axis=1)
 )
 [['color', 'texto']]
)

In [ ]:
dir_back_orders = '../transformed_data/back_orders'
n_cols_to_bold = 3

bytes_data, filename = ef.save_and_download_excel_file(
    reporte, 
    dir_back_orders, 
    file_name,
    sheet_name='Reporte Back Order',
    n_cols_to_bold=n_cols_to_bold,
    return_data=True
)

In [ ]:
# Ahora tenemos que colorear del color específico
# Abrimos el archivo creado
wb = xl.load_workbook(f'{dir_back_orders}/{filename}', data_only=True)
ws = wb.active

rows, cols = reporte.shape

for row, colors in enumerate(colores.values):
    row += 2
    color_celda, color_texto = colors
    fill_color = PatternFill(start_color=f'{color_celda}', end_color=f'{color_celda}', fill_type='solid')
    text_color = Font(color=color_texto)
    for col in range(n_cols_to_bold+1, cols+1):
        ws.cell(row=row, column=col).fill = fill_color
        ws.cell(row=row, column=col).font = text_color
        
wb.save(f'{dir_back_orders}/{filename}')

In [21]:
def load_latest_data(current_dir='..', as_dir=False):
    '''
    Esta función carga los datos más actuales y los regresa en un diccionario
    con los dataframes correspondientes.
    '''
    dir_forecast = f'{current_dir}/transformed_data/forecasts/'
    dir_inventarios = f'{current_dir}/transformed_data/inventarios/'
    dir_transfers = f'{current_dir}/transformed_data/transferencias/'
    dir_back_orders = f'{current_dir}/transformed_data/back_orders'

    latest_forecast = encontrar_forecast_mas_reciente(os.listdir(dir_forecast))
    latest_inventario = encontrar_archivo_mas_reciente(os.listdir(dir_inventarios))
    latest_transfer = encontrar_archivo_mas_reciente(os.listdir(dir_transfers))

    forecast = pd.read_excel(f'{dir_forecast}/{latest_forecast}').drop(['descripcion'], axis=1)
    inventario = pd.read_excel(f'{dir_inventarios}/{latest_inventario}')[['sku', 'inventario']]
    transfers = pd.read_excel(f'{dir_transfers}/{latest_transfer}')[['sku', 'total']]

    clasificaciones = pd.read_excel('../static_data/clasificacion_abc.xlsx').drop(['planta'], axis=1)
    
    # Transformamos el forecast y guardamos los nombres de las columnas
    forecast = (forecast
     .set_index('sku')
     .assign(total_forecast=lambda df: df.sum(axis=1))
    )
    if as_dir:
        resp = {
            'forecast':forecast,
            'inventario':inventario,
            'transfers':transfers,
            'clasificaciones':clasificaciones
        }
    else:
        resp = (forecast, inventario, transfers, clasificaciones)
    return resp

In [28]:
def process_back_orders(forecast, inventario, transfers, clasificaciones):
    '''
    Esta función recibe los dataframes necesarios y genera el reporte de backorders.
    No asigna ningún orden en especial.
    '''
    reporte = (transfers
     .rename(columns={'total':'transferencias'})
     .merge(inventario, on='sku', how='outer')
     .merge(forecast, on='sku', how='left')
     .merge(clasificaciones, on='sku', how='left')
     # Quitamos los productos que no queremos
     .query('inventario > 0 and (clasificacion == "A" or clasificacion == "B") and total_forecast > 0') #
     # Sumamos inventarios y transferencias
     .fillna({'transferencias':0})
     .assign(
         existencias=lambda x: x.transferencias + x.inventario,
         transferencias=lambda x: x.transferencias.replace(0, np.nan)
     )
    )
    return reporte

In [95]:
# Primero cargamos los datos
loaded_data = load_latest_data(current_dir='..', as_dir=False)
# Procesamos el reporte
reporte = process_back_orders(*loaded_data)
# Asignamos los colores correspondientes y ordenamos
# Primero clasificamos por escasez absoluta
# Inicializamos el escalador MinMaxScaler

scaler = MinMaxScaler()


# Calculamos la transformación y la asignamos a una nueva columna 'diferencia_escalada'
(reporte
 .assign(
     diferencia=lambda x: x.existencias - x.total_forecast,
     suficientes=lambda x: (x.diferencia >= 0).astype(int),
     urgencia=lambda df: np.abs(df.groupby('suficientes').diferencia.transform(
         lambda x: scaler.fit_transform(x.values.reshape(-1,1)).reshape(-1)
     )-1)
 )
 .query('diferencia < 0') 
 .sort_values('diferencia')
)

,sku,transferencias,inventario,august 2023,september 2023,total_forecast,descripcion,clasificacion,existencias,diferencia,suficientes,urgencia
17,H006,314.0,1456.0,12519.0,1942.0,14461.0,LAFAM 2X10 LT BV,A,1770.0,-12691.0,0,1.000000
69,P017,1.0,2207.0,6531.0,7872.0,14403.0,PINTO SACO 20 KG,A,2208.0,-12195.0,0,0.960889
54,L094,1151.0,6464.0,11712.0,7517.0,19229.0,FOLEY MAX 1.5% SACO 25 KG,A,7615.0,-11614.0,0,0.915076
21,H044,716.0,1324.0,5715.0,4891.0,10606.0,DRAGOCSON 15X900 ML,A,2040.0,-8566.0,0,0.674736
219,H073,NaN,1623.0,3574.0,3770.0,7344.0,LAFAM 4X5 LT BV,A,1623.0,-5721.0,0,0.450402
...,...,...,...,...,...,...,...,...,...,...,...,...
74,P134,NaN,260.0,157.0,122.0,279.0,GRANUFOS 5% 20 KG,B,260.0,-19.0,0,0.000789
45,INL001,60.0,614.0,497.0,196.0,693.0,AVALANCH 15X1,A,674.0,-19.0,0,0.000789
137,L049,NaN,189.0,138.0,68.0,206.0,DIAZINON 600 4X5 LT,B,189.0,-17.0,0,0.000631
4,E048,144.0,144.0,146.0,159.0,305.0,FITOKLOR 15X1/2 KG,B,288.0,-17.0,0,0.000631


In [13]:
# Ahora vamos a hacerlo en una sola función que haga todo
def genera_reporte_back_order():
    ef = ExcelFunctions()

    dir_forecast = '../transformed_data/forecasts/'
    dir_inventarios = '../transformed_data/inventarios/'
    dir_transfers = '../transformed_data/transferencias/'
    dir_back_orders = '../transformed_data/back_orders'

    latest_forecast = encontrar_forecast_mas_reciente(os.listdir(dir_forecast))
    latest_inventario = encontrar_archivo_mas_reciente(os.listdir(dir_inventarios))
    latest_transfer = encontrar_archivo_mas_reciente(os.listdir(dir_transfers))

    forecast = pd.read_excel(f'{dir_forecast}/{latest_forecast}').drop(['descripcion'], axis=1)
    inventario = pd.read_excel(f'{dir_inventarios}/{latest_inventario}')[['sku', 'inventario']]
    transfers = pd.read_excel(f'{dir_transfers}/{latest_transfer}')[['sku', 'total']]

    clasificaciones = pd.read_excel('../static_data/clasificacion_abc.xlsx').drop(['planta'], axis=1)

    # Transformamos el forecast y guardamos los nombres de las columnas
    forecast = (forecast
     .set_index('sku')
     .assign(total_forecast=lambda df: df.sum(axis=1))
    )

    file_name = f"reporte_back_order_{datetime.now().date().strftime('%d_%m_%Y')}"
    
    # Sumamos las transferencias y los inventarios por producto
    output_order = ['sku', 'descripcion', 'clasificacion', 'inventario', 'transferencias', 'existencias'] + list(forecast.columns)
    threshold = .2

    reporte = (transfers
     .rename(columns={'total':'transferencias'})
     .merge(inventario, on='sku', how='outer')
     .merge(forecast, on='sku', how='left')
     .merge(clasificaciones, on='sku', how='left')
     # Quitamos los productos que no queremos
     .query('inventario > 0 and (clasificacion == "A" or clasificacion == "B") and total_forecast > 0') #
     # Sumamos inventarios y transferencias
     .fillna({'transferencias':0})
     .assign(
         existencias=lambda x: x.transferencias + x.inventario,
         transferencias=lambda x: x.transferencias.replace(0, np.nan),
         porcentaje=lambda x: x.existencias / x.total_forecast - 1
     )
     .sort_values('porcentaje')
     [output_order]
    )

    # El color de este criterio es escacez relativa
    colores = (reporte
     .assign(
         porcentaje=lambda x: x.existencias / x.total_forecast - 1, # Mientras más cercano a -1 es más urgente 
         color=lambda x: x.apply(asigna_color_celda, axis=1),
         texto=lambda x: x.apply(obtener_contraste, axis=1)
     )
     [['color', 'texto']]
    )
    
    n_cols_to_bold = 3

    bytes_data, filename = ef.save_and_download_excel_file(
        reporte, 
        dir_back_orders, 
        file_name,
        sheet_name='Reporte Back Order',
        n_cols_to_bold=n_cols_to_bold,
        return_data=True
    )
    
    # Ahora tenemos que colorear del color específico
    # Abrimos el archivo creado
    wb = xl.load_workbook(f'{dir_back_orders}/{filename}', data_only=True)
    ws = wb.active

    rows, cols = reporte.shape

    for row, colors in enumerate(colores.values):
        row += 2
        color_celda, color_texto = colors
        fill_color = PatternFill(start_color=f'{color_celda}', end_color=f'{color_celda}', fill_type='solid')
        text_color = Font(color=color_texto)
        for col in range(n_cols_to_bold+1, cols+1):
            ws.cell(row=row, column=col).fill = fill_color
            ws.cell(row=row, column=col).font = text_color

    wb.save(f'{dir_back_orders}/{filename}')
    return {
        'Forecast': latest_forecast,
        'Inventario': latest_inventario,
        'Transferencias': latest_transfer,
        'filename': filename
    }

In [14]:
genera_reporte_back_order()

{'Forecast': 'forecast_august_2023_september_2023.xlsx',
 'Inventario': 'inventario_01_08_2023.xlsx',
 'Transferencias': 'transferencias_plantas_01_08_2023.xlsx',
 'filename': 'reporte_back_order_08_08_2023.xlsx'}